In [101]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string
from fim import fpgrowth, apriori
import spacy
from spacy.lang.en import stop_words
nlp = spacy.load('en_core_web_sm')
from ortools.linear_solver import pywraplp

In [102]:
data = {
    "review": [
        "The acting was incredible and the story was engaging.",
        "Terrible plot and boring characters ruined the movie.",
        "Amazing cinematography and a powerful message.",
        "The story was confusing and the acting felt fake.",
        "Outstanding performances and a beautiful soundtrack.",
        "Weak story with predictable twists and flat acting.",
        "The direction was brilliant and the characters were deep.",
        "Poor editing and bad dialogue made it hard to watch.",
        "It was a visually stunning film with great acting.",
        "Nothing made sense and the plot was a mess.",
        "The characters were likable and the story flowed well.",
        "Boring pacing and emotionless performances throughout.",
        "A touching story and breathtaking cinematography.",
        "The movie lacked depth and was hard to follow.",
        "Great story with strong performances from the lead actors.",
        "Terrible direction and overused clichés ruined it.",
        "Solid plot, good pacing, and relatable characters.",
        "I hated the acting and the story was awful.",
        "Excellent script and engaging emotional arc.",
        "The film was empty and the characters were annoying.",
        "Good acting but the plot was weak.",
        "The story was heartfelt and the cast was amazing.",
        "Flat performances and boring script.",
        "Wonderful use of visuals and a solid plot.",
        "Nothing new, just a generic story and poor acting.",
        "Emotional journey with powerful performances.",
        "The characters lacked depth and purpose.",
        "Strong direction and great chemistry between leads.",
        "The dialogue was unrealistic and the story forgettable.",
        "A meaningful plot and stunning cinematography.",
        "Terribly written characters and weak direction.",
        "Great visuals and a fast-paced plot.",
        "Boring from start to finish with no real climax.",
        "Amazing acting saved the otherwise dull story.",
        "The plot went nowhere and the film felt lazy.",
        "Cinematic beauty with a compelling story.",
        "Forgettable characters and forced drama.",
        "Superb acting and flawless direction.",
        "The movie was repetitive and dragged on.",
        "Emotional depth and a well-crafted plot.",
        "The acting felt forced and the story was bad.",
        "Powerful message and top-tier performances.",
        "The movie was disjointed and poorly edited.",
        "Beautiful visuals and strong emotional pull.",
        "Awkward pacing and bland characters.",
        "Loved the story and the acting was great.",
        "Overacted and had no coherent story.",
        "Charming characters and a wonderful plot.",
        "Bad acting and an even worse story.",
        "The plot had depth and the characters felt real.",
        "I was bored by the slow plot and dull acting.",
        "It had heart, humor, and great writing.",
        "The film lacked creativity and charm.",
        "Gripping story with excellent direction.",
        "Predictable story and uninspired acting.",
        "The acting was real and the emotions hit hard.",
        "Flat characters and a recycled plot.",
        "Stunning performances and a rich story.",
        "The film was shallow and the acting was bad.",
        "A compelling story and thoughtful writing.",
        "The movie felt empty and forgettable.",
        "Sharp writing and nuanced performances.",
        "The story was weak and full of holes.",
        "Visually beautiful and emotionally powerful.",
        "The characters were dull and uninteresting.",
        "Excellent performances and a meaningful arc.",
        "Bad script and poor production quality.",
        "Great pacing, solid story, and heartfelt moments.",
        "The film lacked focus and dragged.",
        "The plot was tight and emotionally resonant.",
        "Overly long and lacking in substance.",
        "A story worth telling with actors who delivered.",
        "The characters didn’t evolve and the plot stalled.",
        "It had beautiful direction and strong acting.",
        "The story was derivative and boring.",
        "Heartfelt storytelling with emotional depth.",
        "Uninspired dialogue and bland acting.",
        "The film delivered powerful performances and message.",
        "The acting felt wooden and the story had no soul.",
        "A strong cast and a solid emotional journey.",
        "Forgettable plot and messy storytelling.",
        "The characters felt authentic and the story worked.",
        "It was boring and the acting was weak.",
        "Rich storytelling and great performances.",
        "It lacked any memorable moments or performances.",
        "Masterful direction and impactful acting.",
        "No structure, bad pacing, and forgettable characters.",
        "Beautiful cinematography with a solid emotional base.",
        "The film felt rushed and poorly planned.",
        "The acting was grounded and the story was layered.",
        "Pointless scenes and random story arcs.",
        "A great ensemble and a touching story.",
        "Unconvincing acting and flat storytelling.",
        "Incredible acting and a compelling plot.",
        "No chemistry and a poorly written story.",
        "Smart writing and emotionally strong performances.",
        "The film was a mess with no direction.",
        "Believable characters and a powerful narrative.",
        "Disjointed plot and stiff acting.",
        "Touching story and excellent character arcs.",
    ]
}

df = pd.DataFrame(data)
print(df.shape)
df.head(100)

(100, 1)


,review
0,The acting was incredible and the story was en...
1,Terrible plot and boring characters ruined the...
2,Amazing cinematography and a powerful message.
3,The story was confusing and the acting felt fake.
4,Outstanding performances and a beautiful sound...
...,...
95,Smart writing and emotionally strong performan...
96,The film was a mess with no direction.
97,Believable characters and a powerful narrative.
98,Disjointed plot and stiff acting.


In [104]:
def tokenize(text):
    if not isinstance(text, str):
        return []
    tokens = text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    return list(set([w for w in tokens if w not in stop_words.STOP_WORDS and w.isalpha()]))

def preprocess(df):
    df.dropna(inplace=True)
    df["transaction"] = df["review"].apply(tokenize)
    transactions = df["transaction"].tolist()
    return transactions

In [105]:
transactions = preprocess(df)
for i, t in enumerate(transactions):
    print(f"{i+1}: {t}")

1: ['story', 'engaging', 'incredible', 'acting']
2: ['ruined', 'movie', 'boring', 'plot', 'terrible', 'characters']
3: ['amazing', 'message', 'powerful', 'cinematography']
4: ['felt', 'fake', 'acting', 'story', 'confusing']
5: ['outstanding', 'performances', 'beautiful', 'soundtrack']
6: ['weak', 'acting', 'predictable', 'story', 'flat', 'twists']
7: ['characters', 'deep', 'brilliant', 'direction']
8: ['bad', 'dialogue', 'hard', 'watch', 'poor', 'editing']
9: ['visually', 'acting', 'film', 'stunning', 'great']
10: ['sense', 'plot', 'mess']
11: ['story', 'flowed', 'likable', 'characters']
12: ['emotionless', 'boring', 'performances', 'pacing']
13: ['story', 'touching', 'breathtaking', 'cinematography']
14: ['follow', 'movie', 'hard', 'lacked', 'depth']
15: ['strong', 'actors', 'performances', 'story', 'lead', 'great']
16: ['ruined', 'direction', 'overused', 'terrible', 'clichés']
17: ['characters', 'good', 'plot', 'solid', 'relatable', 'pacing']
18: ['story', 'awful', 'acting', 'hated']

In [106]:
# Itemsets frecuentes con soporte mínimo de 2
r = fpgrowth(transactions, supp=10)
df = pd.DataFrame(r)
df.columns = ['Itemset', 'Support']

df.sort_values(by='Support', ascending=False, inplace=True)
df

,Itemset,Support
0,"(story,)",31
2,"(acting,)",25
3,"(plot,)",18
4,"(characters,)",17
5,"(performances,)",13
1,"(acting, story)",12


In [107]:
r = fpgrowth(transactions, target='r', supp=1, report='aSc')
df = pd.DataFrame(r)
df.columns = ['Consecuente', 'Antecedente', 'Freq', 'Freq(%)', 'Conf']
df.sort_values(by='Conf', ascending=False, inplace=True)
df

,Consecuente,Antecedente,Freq,Freq(%),Conf
0,acting,"(film, great)",1,1.0,1.0
1,story,"(felt, acting)",3,3.0,1.0
2,felt,"(film, plot)",1,1.0,1.0
3,performances,"(strong, story)",1,1.0,1.0
4,great,"(strong, story)",1,1.0,1.0
...,...,...,...,...,...
3390,story,"(overacted, coherent)",1,1.0,1.0
3391,coherent,"(overacted, story)",1,1.0,1.0
3392,overacted,"(coherent, story)",1,1.0,1.0
3393,coherent,"(overacted,)",1,1.0,1.0


In [108]:
# Convertir las reglas a estructura manejable
rules = fpgrowth(transactions, target='r', supp=1, report='aSc')
rule_data = [
    {"ante": r[0], "cons": r[1], "support": r[2], "conf": r[3]}
    for r in rules if r[3] >= 0.5  # opcional: filtrar confianza mínima
]

solver = pywraplp.Solver.CreateSolver("SCIP")
x = [solver.BoolVar(f"x_{i}") for i in range(len(rule_data))]

# Maximizar la suma de la confianza de las reglas seleccionadas
solver.Maximize(solver.Sum(rule_data[i]["conf"] * x[i] for i in range(len(rule_data))))

# Restringimos a seleccionar máximo 5 reglas
solver.Add(solver.Sum(x) <= 10)

# Resolver
status = solver.Solve()

# Mostrar reglas seleccionadas
if status == pywraplp.Solver.OPTIMAL:
    print("=== Reglas seleccionadas ===")
    for i in range(len(rule_data)):
        if x[i].solution_value() > 0:
            r = rule_data[i]
            print(f"{r['ante']} => {r['cons']} | Soporte: {r['support']}%, Confianza: {r['conf']}")
else:
    print("No se encontró solución óptima.")

=== Reglas seleccionadas ===
story => ('felt', 'acting') | Soporte: 3%, Confianza: 3.0
story => ('touching',) | Soporte: 3%, Confianza: 3.0
powerful => ('message',) | Soporte: 3%, Confianza: 3.0
acting => ('predictable', 'story') | Soporte: 2%, Confianza: 2.0
acting => ('predictable',) | Soporte: 2%, Confianza: 2.0
excellent => ('arc',) | Soporte: 2%, Confianza: 2.0
plot => ('wonderful',) | Soporte: 2%, Confianza: 2.0
emotional => ('journey',) | Soporte: 2%, Confianza: 2.0
acting => ('incredible',) | Soporte: 2%, Confianza: 2.0
acting => ('uninspired',) | Soporte: 2%, Confianza: 2.0
